Lab02-1
======


###  Context
#### Regression(Linear Regression)
   + Univariate regression(단변량 회귀분석)
   + Multivariate regression(다변량 회귀분석)

# Regression ( 회귀 분석 )
+ 어떤 연속적인 값을 갖는 Target변수가 그 외 다른 변수(features)에 의해 설명된다고 보고, 그 함수 관계를 조사하는 기법입니다.


\- Target변수 <영향을 받는 변수(Y)> : 
<br>반응변수(response variable), __종속변수(dependent variable)__, 결과변수(outcome variable)


\- 그외 다른 변수 <영향을 주는 변수(X)> : 
<br>설명변수(explanatory variable), __독립변수(independent variable)__, 예측 변수(predictor variable) 


## Linear regression
+ 회귀 모델이 회귀 계수들의 선형 결합(Linear Relationship)만으로 이루어진 함수임을 말합니다.

### - Linear Relationship
+ ### Hyphothesis(가설공간)

     $Y = B_0 + B_1X_1 + B_2X_2+ \cdot\cdot\cdot B_dX_d$
    ($Y$: Target,  $X_i$ : features,  $d$ : # of features, $B_i$ : model parameters)


+ 회귀 분석을 통해 추정되는 계수의 크기 및 방향성(+,-)를 통해 독립 변수가 종속 변수에 미치는 영향을 알 수 있게 됩니다.
    
### - How to find relationship?

+ ### Least squares method(최소제곱법)<br>

    + 변수 X와 Y의 관계를 가장 잘 나타낸다는 것은 곧 데이터가 나타내는 점들에 가장 가까이 있는 직선을 찾는다는 것입니다. 따라서 이는 곧 모든 점들과의 거리의 합이 최소가 되는 직선을 찾는 것이라고 할 수 있습니다.
    
    $Y: 실제 값 \ \ \hat{Y}: 추정 값(Target)$
    <br>$Y = \hat{Y} + e$
    <br>$e = Y - \hat{Y} : 잔차$(각 데이터와 직선 사이의 거리)
    
    $\sum_{i=1}^n \ {e_i^2}: \ Residual \ Sum \  of \ Squares (RSS)$ 
    ### minRSS = min$\sum_{i=1}^n {e_i^2}$

In [ ]:
from os.path import join
import numpy as np

from sklearn import linear_model
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from pandas.plotting import scatter_matrix

## 1. 데이터 살펴보기

+ 이번 실습에서는 당뇨병 환자 데이터1)와 자동차 데이터2)를 사용합니다.

## 1) 예제 : 'diabetesDataset.csv'

데이터를 살펴보기 쉽게 csv 파일을 pandas DataFrame으로 읽습니다.

In [ ]:
df = pd.read_csv(join('data','diabetesDataset.csv'))
print(df.shape)

DataFrame.head() 함수로 처음 5개의 데이터를 살펴볼 수 있습니다.

In [ ]:
df.head()

In [ ]:
for dim in df.columns:
        df[dim] -= np.min(df[dim])
        df[dim] /= np.max(df[dim])

describe() 메서드는 숫자형 특성의 요약 정보를 줍니다.
<br>count(sample 수), mean(평균), min, max, 사분위수(1(25%),2(50%),3(75%))

<br>%(백분위수 : 전체 관측값에서 주어진 백분율이 속하는 하위 부분의 값)
<br>ex  (__bmi__): 0-25% 구역은  -3.422907e-02보다 작거나 같음, 0-50%는 -7.283770e-03보다 작거나 같음, 75%는 3.564384e-02보다 작거나 같음.

In [ ]:
df.describe()

hist() 메서드는 히스토그램을 줍니다. 히스토그램은 주어진 값의 범위(수평축)에 속한 샘플 수(수직축)를 나타냅니다.

In [ ]:
df.hist(bins = 20, figsize=(10, 15))
plt.show()

+ 데이터는 442명의 환자 데이터를 포함하며, 각 환자별로 11개의 특징들을 포함하고 있습니다.


+ Attribute Information:
     - Age
     - Sex
     - Bmi : Body mass index
     - map : Average blood pressure
     - tc, ldl, hdl, tch, ltg, glu : blood serum(혈청) measurements
     - y : a quantitative measure of disease progression one year after baseline

***

## 2) 연습문제 : 'ToyotaDataset.csv'

In [ ]:
# 데이터를 살펴보기 쉽게 csv 파일을 pandas DataFrame으로 읽습니다.
# df와는 다른 이름의 변수를 사용합니다. (df2)
df2 = pd.read_csv(join('data','ToyotaDataset.csv'))
print(df2.shape)

In [ ]:
# DataFrame.head() 함수로 처음 5개의 데이터를 살펴볼 수 있습니다.
df2.head()

In [ ]:
for dim in df2.columns:
        df2[dim] -= np.min(df2[dim])
        df2[dim] /= np.max(df2[dim])

In [ ]:
# describe() 메서드는 숫자형 특성의 요약 정보를 줍니다.
df2.describe()

In [ ]:
# hist() 메서드는 히스토그램을 줍니다. 
df2.hist()
plt.show()

***

# \* 이번 실습에서는 Linear regression을 통해 환자의 1년 후 질병 진행의 정도1)와 자동차의 가격2)을 추정해보도록 하겠습니다.

(Target : y)

## < Simple linear regression(단순회귀분석) >
+ Target 변수에 영향을 미치는 독립변수의 수가 1개일 때를 말합니다.

<center>$Y = B_0 + B_1X_1$

## 2. Scratch로 단변량 회귀식 추정하기

### \- use only one feature(Select bmi)

Data를 input 데이터(diabetes_X)와 output 데이터(Target, diabetes_Y)로 나눕니다.

In [ ]:
# 컬럼이름으로 다중 컬럼 선택 (return DatFrame)
## 하나의 컬럼을 선택할 경우 Series를 반환
## 하나의 컬럼을 DataFrame으로 반환하기 위해서는 컬럼 리스트로 입력

label = 'bmi'
diabetes_X = df[[label]]
diabetes_Y = df[['y']]

In [ ]:
print(diabetes_X.shape)
diabetes_X.head()

In [ ]:
diabetes_Y.head()

### \- plot the data point

In [ ]:
#plot
tmpx = diabetes_X.values.tolist()
tmpy = diabetes_Y.values.tolist()

plt.scatter(tmpx, tmpy, color='blue', label='Data Point(patients)')

# x축 label
plt.xlabel(label)

# y축 label
plt.ylabel('y')
plt.legend()# 범례
plt.show()

### \- find relationship

## \* Least squares method(최소제곱법)

### minRSS 
### = min$\sum_{i=1}^n \ {e_i^2}$
### = min$\sum_{i=1}^n ({y_i - \hat{y_i}})^2$
### = min$\sum_{i=1}^n ({y_i - \hat{\beta_0} - \hat{\beta_1}x_i})^2$

### \- $ \hat{\beta_0}$ \& $\hat{\beta_1}$는 어떻게 찾는가?

Convex function 이므로 미분을 통해 최소값을 계산합니다.

### \< Explicit solution \>
### $\hat{\beta_1} = \frac{\sum_{i=1}^n {(X_i-\bar{X_i})*(Y_i-\bar{Y})}}{\sum_{i=1}^n ({X_i-\bar{X_i}})^2}$

### $\hat{\beta_0} = \bar{Y} - \hat{\beta_1}*\beta_1 \bar{X}$

In [ ]:
# inputs 과 outputs 변수(X, Y) 초기화
X = diabetes_X.values.tolist()
Y = diabetes_Y.values.tolist()

# inputs의 평균
x_mean = np.mean(X)

# outputs의 평균
y_mean = np.mean(Y)

# input의 총 수
n = len(X)

# 모델 파라미터 b1, b0 계산
numerator = 0 #분자
denominator = 0 #분모
for i in range(n):
    numerator += (X[i] - x_mean) * (Y[i] - y_mean)
    denominator += (X[i] - x_mean) ** 2 # ** : 제곱
    
b1 = numerator / denominator
b0 = y_mean - (b1 * x_mean)

# 모델 파라미터 출력
print('b1:',b1)
print('bo:',b0)
print('y=',b0[0],'+',b1[0],' * x')

### \- result

In [ ]:
# plot

x_max = np.max(X)
x_min = np.min(X)

# 회귀식으로 데이터의 결과 추정위한 독립변수
x = np.linspace(x_min, x_max, 1000)

# 회귀식으로 데이터의 결과 추정
y = b0 + b1 * x

# 회귀 결과 plot
plt.plot(x, y, color='red', label='Linear Regression')

# 데이터 좌표 plot
plt.scatter(X, Y, color='black', label='Data Point(patients)')

# x축 label
plt.xlabel(label)

# y축 label
plt.ylabel('y')
plt.legend()
plt.show()

### -evaluation

### \- 모델이 주어진 데이터에 얼마나 적합한가?

\* Root Mean Squared Error(RMSE)



+ 각 점들과 직선과의 거리 제곱 평균의 제곱근
+ ### $RMSE = \sqrt{\sum_{i=1}^m \frac{1}{m}{(\hat{y_i}-y_i)^2}}$

Mean Squared Error(MSE): 추정값$(\hat{y_i})$과 실제값$(y_i)$의 차이의 제곱의 평균
+ 분산( 평균과 실제값의 차이의 제곱의 평균 )과 유사

In [ ]:
# input 총 수
n = len(X)

rmse = 0
for i in range(n):
    y_pred=  b0 + b1* X[i]
    rmse += (Y[i] - y_pred) ** 2
    
rmse = np.sqrt(rmse/n)
print(rmse)

\* $R^2$ scrore (결정 계수)

+ 종속 변수의 총 변화량 중 모델(추정된 회귀식)로 설명 가능한 부분(변화량)의 비율을 의미합니다.

![R2](./images/R2.png)


SST : total sum of squares (모든 개체가 가지는 총 변화량)

SSR : sum of squares residuals (총 변화량 중 회귀식으로 설명 가능한 변화량)

SSE : sum of squares error(총 변화량 중 회귀식으로는 설명 불가능한 변화량)

(즉, SST = SSR + SSE)

+ ### $R^2 = \frac{SSR}{SST} = \frac{\sum(\hat{y_i}-\bar{y})^2}{\sum(y_i-\bar{y})}$

  ###  $= 1 - \frac{SSE}{SST} = 1 - \frac{\sum({y_i}-\hat{y})^2}{\sum(y_i-\bar{y})}$


$0<R^2<1$ : 1에 가까울수록 높은 설명력을 보인다고 할 수 있습니다.

$R^2 = 1$

:the regression model fits perfectly

$R^2 = 0$

:the regression model does not explain anything about the relationship between X and Y

![RegrR^2](./images/RegrR^2.png)

In [ ]:
sumofsquares = 0
sumofresiduals = 0
# input의 총 수
n = len(X)

for i in range(n) :
    y_pred = b0 + b1 * X[i]
    sumofsquares += (Y[i] - y_mean) ** 2
    sumofresiduals += (Y[i] - y_pred) **2
    
score  = 1 - (sumofresiduals/sumofsquares)
print(score)

+ 계산된 $R^2$값이 약 0.372 이므로 모델은 어느 정도 데이터에 적합하지만 다른 중요한 요소가 존재한다는 것을 의미합니다.

## 3. Scikit-learn으로 단변량 회귀식 추정하기

## 1) 예제

### \- use only one feature(bmi)

(위와 동일)

### \- plot the data point

(위와 동일)

### \- find relationship
+ 앞서 언급한 모델 파라미터 추정을 scikit-learn에서는 estimator(추정기)라는 객체를 기반으로 합니다. 추정 자체는 fit() 메서드에 의해 수행됩니다.

다음은 선형 회귀를 위한 LinearRegression 추정기 입니다.

In [ ]:
# 선형회귀 추정기 생성
lr = linear_model.LinearRegression() 

fit() 메서드를 통해 모델 파라미터를 추정합니다.

In [ ]:
# input 대해 선형 회귀(모델 파라미터 추정)
diabetes_X = diabetes_X.values.tolist()
diabetes_Y = diabetes_Y.values.tolist()
lr.fit(diabetes_X, diabetes_Y)

In [ ]:
# 모델 파라미터 출력
print('Model parameters: \n')
print('b1:', lr.coef_[0,0])
print('b0:', lr.intercept_[0])
print()
print('y=',lr.intercept_[0],'+',lr.coef_[0,0],'* x')

계산된 모델 파라미터를 적용하여 데이터 세트에 대해 추정값을 반환합니다.

In [ ]:
# 회귀식으로 데이터의 결과 추정
diabetes_y_pred = lr.predict(diabetes_X)

### \- result

In [ ]:
# Plot

# 데이터 좌표 plot
plt.scatter(diabetes_X, diabetes_Y,  color='black', label='Data Point(patients)')

# 회귀 결과 plot
plt.plot(diabetes_X, diabetes_y_pred, color='blue', linewidth=3, label='Linear Regression')

# x축 label
plt.xlabel(label)

#y축 label
plt.ylabel('y')
plt.legend()
plt.show()

### \- evaluation

In [ ]:
# R2 scroe
print('R2 score: %.2f' % r2_score(diabetes_Y, diabetes_y_pred))

***

## 2) 연습문제

### use only one feature
독립변수와 종속변수를 선택합니다.

In [ ]:
label_test =
dfx = 
dfy = 

In [ ]:
dfx.head()

In [ ]:
dfy.head()

### plot the data point
데이터 점을 산점도 그래프로 나타냅니다.

### find relationship by sklearn

모델 파라미터를 계산하여 출력하고, 데이터에 대해서 회귀 결과를 얻습니다.

### result
결과를 그래프로 나타냅니다.

In [ ]:
# Plot

# 데이터 좌표 plot

# 회귀 결과 plot

# x축 label

# y축 label


### evaluation
R2를 이용하여 테스트 결과를 평가합니다.

***

## < Multiple regression (다중회귀분석) >

+ Target 변수에 영향을 미치는 독립변수의 수가 d개일 때를 말합니다.

<center>$Y = B_0 + B_1X_1 + B_2X_2+ \cdot\cdot\cdot B_dX_d$

## 3. Scikit-learn으로 다변량 회귀식 추정하기

## 1) 예제

### \- use more than two features(Select bmi, tch)

In [ ]:
# 컬럼이름으로 다중 컬럼 선택 (return DatFrame)
## 하나의 컬럼을 선택할 경우 Series를 반환
## 하나의 컬럼을 DataFrame으로 반환하기 위해서는 컬럼 리스트로 입력

label2 = ['bmi', 'tch']
diabetes_X_multi = df[label2]
diabetes_Y_multi = df[['y']]

### \- plot the data point

In [ ]:
# Plot

# x,y축 값 설정 (독립변수)
bmi = diabetes_X_multi[['bmi']]
tch = diabetes_X_multi[['tch']]

fig = plt.figure()
ax = Axes3D(fig)

# 산점도 그래프
ax.scatter(bmi, tch, diabetes_Y_multi, color='red')

# x축 label
ax.set_xlabel('bmi')

# y축 label
ax.set_ylabel('tch')

# z축 label
ax.set_zlabel('y')
plt.show()

### \- find relationship

 다음은 선형 회귀를 위한 LinearRegression 추정기 입니다.

In [ ]:
# 선형회귀 추정기 생성
lr2 = linear_model.LinearRegression()

fit() 메서드를 통해 모델 파라미터를 추정합니다.

In [ ]:
# input 대해 선형 회귀(모델 파라미터 추정)
lr2.fit(diabetes_X_multi, diabetes_Y_multi)

In [ ]:
# 모델 파라미터 출력
print('Model parameters: \n')
print('b2:', lr2.coef_[0,0])
print('b1:', lr2.coef_[0,1])
print('b0:', lr2.intercept_[0])
print()
print('y=',lr2.intercept_[0],'+',lr2.coef_[0,1],'*x1 + ', lr2.coef_[0,0], '*x2')

계산된 모델 파라미터를 적용하여 데이터 세트에 대해 추정값을 반환합니다.

In [ ]:
diabetes_y_pred_multi = lr2.predict(diabetes_X_multi)

### \- result

In [ ]:
# Plot 

# x,y축 값 설정
bmi = diabetes_X_multi[['bmi']]
tch = diabetes_X_multi[['tch']]

# 회귀식으로 결과 추정위한 축 설정
xx1, xx2 = np.meshgrid(np.linspace(diabetes_X_multi.bmi.min(),
                                   diabetes_X_multi.bmi.max(), 100), 
                       np.linspace(diabetes_X_multi.tch.min(), 
                                   diabetes_X_multi.tch.max(), 100))
# matplotlib 3d plot
fig = plt.figure()
ax = Axes3D(fig, azim= -100, elev=50) #azim : 방위각도, azim : 고도 
ax.scatter(bmi, tch, diabetes_Y_multi, color='red')


# 회귀식으로 결과(hyperplane) 추정
Z =lr2.intercept_[0] +lr2.coef_[0,0] * xx1 + lr2.coef_[0,1] * xx2

# plot hyperplane
ax.plot_surface(xx1,xx2,Z)
ax.set_xlabel('bmi')
ax.set_ylabel('tch')
ax.set_zlabel('y')

plt.show()

### \- evaluation

+ 총변화량(SST) 중에서 회귀직선에 의해 설명되는 변화량(SSR)의 비율은 독립변수의 수가 증가하면 따라서 커지는 증가함수 형태입니다.(독립변수의 개수에 의존)

\* $R_{adj}^2$ scrore (수정된 결정 계수)

+ 즉, 일반 $R^2$는 활용된 독립변수의 수에 민감하게 작용하므로, 독립변수의 수를 discount해주는 방식의 수정된 $R^2$ score가 지표로 활용됩니다.


+ ### $R_{adj}^2 = 1 - [\frac{n-1}{n-(p+1)}]\frac{SSE}{SST}$
(결정계수보다 항상 작은것이 특징)

In [ ]:
# R2 scroe
R2 = r2_score(diabetes_Y_multi, diabetes_y_pred_multi)
print('R2 score: %.2f' % R2)

In [ ]:
# adj R2 score
n = len(diabetes_X_multi)
p = len(label)

Adj_r2 = 1-(1-R2)*(n-1)/(n-p-1)
print('R2 score: %.2f' % Adj_r2)

+ 계산된 $Adj \ R^2$값이 0.40 이므로 모델은 단변량 모델보다 더 좋은 설명력을 보임을 의미합니다.

---

## 2) 연습문제

### use more than two features
독립변수와 종속변수를 선택합니다.

In [ ]:
dfx_multi.head()

In [ ]:
dfy_multi.head()

### plot the data point
데이터 점을 산점도 그래프로 나타냅니다.

In [ ]:
# Plot

# x,y축 값 설정 (독립변수)

# figure
fig = plt.figure()
ax = Axes3D(fig)

# 산점도 그래프

# x축 label

# y축 label

# z축 label

plt.show()

### find relationship
모델 파라미터를 계산하여 출력하고, test 데이터에 대해서 회귀 결과를 얻습니다.

In [ ]:
# 선형회귀 추정기 생성

# input 대해 선형 회귀(모델 파라미터 추정)

# 모델 파라미터 출력

# 회귀식으로 데이터의 결과 추정


### result
결과를 그래프로 나타냅니다.

In [ ]:
# Plot

# x,y축 값 설정 (독립변수)


# 회귀식으로 결과 추정위한 독립변수 설정(input)

# matplotlib 3d plot

# 회귀식으로 결과 추정

# plot hyperplane


plt.show()

### evaluation
R2를 이용하여 테스트 결과를 평가합니다.

In [ ]:
# R2 scroe
R2 = 
print('R2 score: %.2f' % R2)

In [ ]:
# adj R2 score
Adj_R2 = 
print('R2 score: %.2f' % Adj_r2)

(표본의 크기와 독립변수의 수를 고려하는 $Adj \ R^2$와 $R^2$는 표본이 큰 경우 거의 동일해짐을 확인할 수 있습니다.)

# ** 설명력을 높여보자 **

## 2) 연습문제

### \- sklearn을 활용하여 설명력이 높은 회귀식을 찾아보세요.

In [ ]:
df2 = pd.read_csv(join('data', 'ToyotaDataset.csv'))

In [ ]:
df2.head()

In [ ]:
df2.columns

In [ ]:
# newlabel을 수정해가며 설명력이 높은 회귀식을 찾아보세요.
newlabel = ['Age']

dfx = df2[newlabel]
dfy = df2[['Price']]

print(dfx.shape)

### regression

In [ ]:
# sample code

## 선형회귀 추정기 생성
# lr3 = linear_model.LinearRegression()

## input 대해 선형 회귀(모델 파라미터 추정)
# lr3.fit(X, Y)

## 모델 파라미터 출력
# print('Model parameters: \n')

# rgstr = ''
# for i in range(len(lr3.coef_[0])):
#     print("b%d" %(len(lr3.coef_[0])-i),": ", "%f" %lr3.coef_[0,i])
#     rgstr = ' + '+repr(lr3.coef_[0,i]) + '*x'+repr(len(lr3.coef_[0])-i) + rgstr

# print('b0:', lr3.intercept_[0])
# print()
# rgstr = repr(lr3.intercept_[0]) + rgstr
# print('y = ', rgstr)

### \- evaluation

In [ ]:
# y_pred = lr3.predict(X_test)
# n = len(X_test)
# p = len(label2)

# R2 = r2_score(y_test, y_pred)
# Adj_r2 = 1-(1-R2)*(n-1)/(n-p-1)

# print('Adj R2 score: %.2f' %  Adj_r2)

---

### Reference

+ Dataset 
#### < diabetesDataset >
Source URL:
http://www4.stat.ncsu.edu/~boos/var.select/diabetes.html

For more information see:
Bradley Efron, Trevor Hastie, Iain Johnstone and Robert Tibshirani (2004) "Least Angle Regression," Annals of Statistics (with discussion), 407-499.
(http://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf)